## <font color='red'>Задание 1.</font>

Реализовать функции генерации ключей, формирования и проверки подписи ЭЦП Эль-Гамаля.
Являются ли данные алгоритмы полиномиальными?

### Дополнительные функции

In [3]:
import random
import sympy as smp

In [2]:
def generate_binary_string(N):
    S = "1"
    for i in range(1, N):
        x = random.randint(0, 1)
        S += str(x)
    return S

### Дополнительные функции из лекции

In [4]:
def OS2IP(X: bytes) -> int:
    x = 0
    for i in range(len(X)):
        x += X[i] * 256**(len(X)-1-i)
    return x  

In [5]:
def I2OSP(x: int, XLen: int) -> bytes:
    if x >= 256**XLen:
        raise ValueError("Number %d must be less than %d" % (x,256**XLen))
    b = []
    for _ in range(XLen):
        b.append(x % 256)
        x >>= 8
    b.reverse()
    return bytes(b)

In [6]:
import math
def Len(n: int)-> int:
    if type(n) == int:
        return math.ceil((len(bin(n))-2)/8)
    s = 0
    for i in n:
        s += Len(i)
    return s    

In [7]:
def H(m: bytes) -> bytes:
    import hashlib
    return hashlib.sha3_256(m).digest()

#### Проверяю

In [7]:
print(generate_binary_string(2))
print(generate_binary_string(3))
print(generate_binary_string(14))

10
100
10001011011110


### Дополнительные функции из предыдущей лабораторной

In [8]:
def is_generator(p, g):
    group_ln = p - 1
    pr_dict = smp.factorint(group_ln)
    for el in pr_dict:
        if pow(g, group_ln // el, p) == 1:
            return False
    return True

In [9]:
def get_generator(p):
    while(True):
        G = range(1, p)
        el = G[random.randint(0, p - 2)]
        if is_generator(p, el):
            return el
    return None

#### Проверяю

In [10]:
pr1 = 107
gen1 = get_generator(pr1)
i = 1
G1 = []
while True:
    temp_pow = pow(gen1, i, pr1)
    G1 += [temp_pow]
    if (temp_pow == 1):
        break
    i += 1
print(sorted(G1))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106]


### Генерация ключей

In [10]:
import random;
import sympy as smp

def get_key(L):
    p = int(str(generate_binary_string(L)), 2)
    while not smp.isprime(p):
        p = int(str(generate_binary_string(L)), 2)
    g = get_generator(p)
    x = random.randint(2, p - 2)
    y = pow(g, x, p)
    return [(y, g, p), x]

#### Пошаговая проверка алгоритма по частям

In [275]:
temp_L_tupl = 5, 6, 7, 8, 12, 45, 466
for temp_L in temp_L_tupl:
    p = int(str(generate_binary_string(temp_L)), 2)
    while not smp.isprime(p):
        p = int(str(generate_binary_string(temp_L)), 2)
    print(p, smp.isprime(p))

29 True
43 True
73 True
233 True
2741 True
20470548801511 True
156658619174199887069228386554651052402181304373488409362239466690616384921833834284260512824087499294130779152364952892856064165230429981911 True


### Генерации подписи

In [18]:
import math

def get_sign(k_e, k_d, m):
    h = OS2IP(H(m))
    k = random.randint(2, k_e[2] - 2)
    check = math.gcd(k, k_e[2])
    while(check != 1):
        k = random.randint(2, k_e[2])
        check = math.gcd(k, k_e[2])
    a = pow(k_e[1], k, k_e[2])
    b = pow(k, -1, k_e[2]) * (h - k_d * a) % (k_e[2] - 1)
    return (a, b)

#### Пошаговая проверка алгоритма по частям

In [19]:
temp_L = 3
k_e1, k_d1 = get_key(temp_L)
print("k_e =", k_e1, "k_d1 =", k_d1)
k = random.randint(2, k_e1[2] - 2)
check = math.gcd(k, k_e1[2])
while(check != 1):
    k = random.randint(2, p - 2)
print("k =", k)

k_e = (6, 5, 7) k_d1 = 3
k = 3


In [20]:
temp_L = 107
temp_msg = b'hello'
k_e1, k_d1 = get_key(temp_L)
print("k_e =", k_e1, "k_d1 =", k_d1)
temp_sign = get_sign(k_e1, k_d1, temp_msg)
print("a =", temp_sign[0], "b =", temp_sign[1])

k_e = (57661937530506537733170772440365, 15174890028097398222691798689716, 101806037003831403807449321480377) k_d1 = 27251254360664482306284380225605
a = 15446560569971982622460312720386 b = 64824974504191881007490374466408


### Проверка подписи

In [21]:
def check_sign(k_e, m, s):
    h = OS2IP(H(m))
    # print((pow(k_e[0], s[0], k_e[2]) * pow(s[0], s[1], k_e[2])) % k_e[2])
    # print(pow(k_e[1], h, k_e[2]))
    return (((pow(k_e[0], s[0], k_e[2]) * pow(s[0], s[1], k_e[2])) % k_e[2]) == (pow(k_e[1], h, k_e[2]) % k_e[2]))

#### Пошаговая проверка алгоритма по частям

In [347]:
temp_L = 5
temp_msg = b'hello'
k_e1, k_d1 = get_key(temp_L)
print("k_e =", k_e1, "k_d1 =", k_d1)
temp_sign = get_sign(k_e1, k_d1, temp_msg)
print("a =", temp_sign[0], "b =", temp_sign[1])
check_sign(k_e1, temp_msg, temp_sign)

k_e = (20, 21, 23) k_d1 = 19
a = 5 b = 19
1
16


False

### Измененный алгоритм

In [49]:
def get_sign(k_e, k_d, m):
    h = OS2IP(H(m))
    k = random.randint(2, k_e[2] - 2)
    check = math.gcd(k, k_e[2])
    while(check != 1):
        k = random.randint(2, k_e[2])
        check = math.gcd(k, k_e[2])
    a = pow(k_e[1], k, k_e[2])
    b = pow(k_e[0], k, k_e[2]) * h % k_e[2]
    return (a, b)

def check_sign(k_e, m, s, k_d):
    h = OS2IP(H(m))
    return (s[1] * pow(pow(s[0], k_d, k_e[2]), -1, k_e[2]) % k_e[2]) == (h % k_e[2])

#### Проверка измененного алгоритма

In [52]:
temp_L = 107
temp_msg = b'hello, Andrey'
k_e1, k_d1 = get_key(temp_L)
print("k_e =", k_e1, "k_d1 =", k_d1)
temp_sign = get_sign(k_e1, k_d1, temp_msg)
print("a =", temp_sign[0], "b =", temp_sign[1])
check_sign(k_e1, temp_msg, temp_sign, k_d1)

k_e = (68411123238970166501289602525466, 76218450256160721109785973441270, 143601346624513385625811616644439) k_d1 = 62812943687084426422530738144643
a = 107005018866138429659269013750273 b = 15900100774542704958795670405285


True

#### Проверка по сценарию

In [371]:
pseudo_p = 107
pseudo_g = 2
pseudo_x = 67
pseudo_y = pow(pseudo_g, pseudo_x, pseudo_p)
print(pseudo_y)

temp_msg = b'B'
pseudo_k = 45
h = 66
pseudo_a = pow(pseudo_g, pseudo_k, pseudo_p)
pseudo_b = h * pow(pseudo_y, pseudo_k, pseudo_p) % pseudo_p
print(pseudo_a, pseudo_b)

pseudo_b * pow(pseudo_a, -1 * pseudo_x, pseudo_p) % pseudo_p

94
28 9


66

In [40]:
pseudo_p = 11
pseudo_g = 2
pseudo_x = 8
pseudo_y = pow(pseudo_g, pseudo_x, pseudo_p)
print(pseudo_y)

pseudo_k = 9
h = 23455423 
pseudo_a = pow(pseudo_g, pseudo_k, pseudo_p)
pseudo_b = h * pow(pseudo_y, pseudo_k, pseudo_p) % pseudo_p
print(pseudo_a, pseudo_b)

print(pseudo_b * pow(pseudo_a, -1 * pseudo_x, pseudo_p) % pseudo_p)

print(h % pseudo_p)

3
6 8
2
2


## <font color='red'>Задание 4.</font>

Реализовать функции генерации ключей, формирования и проверки подписи ЭЦП Шнорра. Являются ли данные алгоритмы полиномиальными?

In [17]:
def get_Shnorr_key(L, N):
    q = int(str(generateBinaryString(N)), 2)
    p = int(str(generateBinaryString(L)), 2)
    while ((p - 1) % q != 0):
        p = int(str(generateBinaryString(L)), 2)
        q = int(str(generateBinaryString(N)), 2)
    g = get_generator(range(1, p), p)
    while (pow(g, int((p - 1) / q), p) == 1):
        g = get_generator(range(1, p), p)
    g = pow(g, int((p - 1) / q), p)
    x = random.randint(1, q)
    y = pow(g, x, p)
    return (y, g, p, q), x

In [30]:
def get_Shnorr_sign(k_e, k_d, m):
    k = random.randint(1, q)
    a = pow(g, k, p)
    h = H(str(a + m))
    b = (k - k_d * h) % q
    return b, h

In [13]:
def sign_Shnorr_check(k_e, m, s):
    a = (pow(k_e[1], s[0], p) * pow(k_e[0], s[1])) % p
    return h == H(a // m) % q

## <font color='red'>Задание 5.</font>
ЭЦП Шнорра задана параметрами $\;g,\;p,\; q.\;$  Для документа $\;m\;$ получена подпись $\;s=(b,h).\;$  Убедиться, что данная подпись подходит для любого открытого ключа $\;y.\;$ Хорошо это или плохо характеризует ЭЦП Шнорра с криптографической точки зрения? Какая особенность подписи повлияла на такой результат? 

In [14]:
m = b'Document'
g = 9516562544564
p = 26443632403687
q = 577259
b = 545127
h = 3316258630769272128572666739707468991127113456633944108576235413081343728178

In [18]:
k_e, k_d = get_Shnorr_key(Len(p), Len(q))

In [19]:
k_e

(1, 6, 37, 4)

In [20]:
k_d

4

In [21]:
import math
def Len(n: int)-> int:
    if type(n) == int:
        return math.ceil((len(bin(n))-2)/8)
    s = 0
    for i in n:
        s += Len(i)
    return s    
number = 1000

In [31]:
sign = get_Shnorr_sign(k_e, k_d, m)

TypeError: unsupported operand type(s) for +: 'int' and 'bytes'